<a href="https://colab.research.google.com/github/aliswh/lastfm/blob/main/batch_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/aliswh/lastfm
!cd lastfm; pip install -r requirements.txt

Cloning into 'lastfm'...
remote: Enumerating objects: 330, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 330 (delta 203), reused 222 (delta 103), pack-reused 0
Receiving objects: 100% (330/330), 2.17 MiB | 2.24 MiB/s, done.
Resolving deltas: 100% (203/203), done.
     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 198 kB 55.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=573cfb3a224779d38b65f5a588a4e8d82d322823790e89fd83ef291d4a778b55
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from lastfm.src.ingestion_layer.googlestorage import *
from lastfm.src.ingestion_layer.pylastsource import *
from lastfm.src.ingestion_layer.batchwriter import *
from lastfm.src.ingestion_layer.config import *

In [3]:
storage = GoogleStorageJSON('./lastfm/src/ingestion_layer/creds.json',BUCKET_NAME)
source = PyLastSource(AUTH_DATA)
writer = BatchWriter(source, storage)

In [5]:
#storage.delete('')

In [ ]:
#Batch job
writer.write('', SEED_USER, 5, 20, debug=True)

[Global top tags] {'rock': 4003211, 'electronic': 2402787, 'seen live': 2145524, 'alternative': 2105763, 'indie': 2032023, 'pop': 2002054, 'female vocalists': 1609862, 'metal': 1269878, 'alternative rock': 1177215, 'jazz': 1162934, 'classic rock': 1141844, 'ambient': 1073716, 'experimental': 1068145, 'folk': 927848, 'punk': 892814, 'indie rock': 887915, 'Hip-Hop': 881618, 'hard rock': 865618, 'black metal': 857862, 'instrumental': 851689, 'singer-songwriter': 836806, 'dance': 803344, '80s': 782430, 'Progressive rock': 728235, 'death metal': 726454, 'heavy metal': 708744, 'hardcore': 697339, 'british': 688950, 'soul': 678624, 'chillout': 648513, 'electronica': 632160, 'rap': 583084, 'industrial': 566876, 'Classical': 566682, 'Soundtrack': 562103, 'punk rock': 554058, 'blues': 547183, 'thrash metal': 498508, '90s': 486219, 'metalcore': 478523, 'acoustic': 477686, 'psychedelic': 475687, 'japanese': 464545, 'post-rock': 443657, 'hip hop': 438421, 'Progressive metal': 436440, 'german': 4352

In [16]:
recent_tracks = storage.read('recent_tracks',dir=True)
tracks_list = storage.read('testing_branch/tracks',dir=True)
global_top_tags = storage.read('testing_branch/global_top_tags',dir=False)
global_top_tags = set(list(global_top_tags)[:10])
tracks_rdd = sc.parallelize(tracks_list)
print(tracks_rdd.take(1))
tags_rdd = tracks_rdd.map(lambda x: x['tags'].items())\
                     .flatMap(lambda x:x)\
                     .reduceByKey(add)\
                     .reduce(lambda x,y: topN(x,y,1,reverse=True,top=10))

print(tags_rdd)

[{'artist': 'Kim Petras', 'title': 'Throat Goat', 'album': None, 'duration': 0, 'tags': {}, 'url': 'https://www.last.fm/music/kim%2bpetras/_/throat%2bgoat', 'id': '0af9c69909f292930e7f66d2cb8ba0db2696ba1e'}]
[('pop', 250), ('Energetic', 209), ('hot', 206), ('electropop', 200), ('bop', 200), ('electro house', 200), ('Playful', 200), ('horny', 200), ('sexual', 200), ('banger', 200)]


SyntaxError: ignored

In [ ]:
for recent_track in recent_tracks:
  print(recent_track)

{'0': {'artist': 'Alina Baraz', 'title': 'More Than Enough', 'date': '16 Feb 2022, 00:17', 'album': 'It Was Divine', 'timestamp': '1644970620', 'id': '997c665d10565914f05320254d0d46b46e734155'}, '1': {'artist': 'Alina Baraz', 'title': 'Make You Feel', 'date': '16 Feb 2022, 00:12', 'album': 'Urban Flora', 'timestamp': '1644970377', 'id': '64704ce20b0403804e51973749c0a7023bb5f1e4'}, '2': {'artist': 'Alina Baraz', 'title': 'Who Got Me', 'date': '16 Feb 2022, 00:09', 'album': 'It Was Divine', 'timestamp': '1644970153', 'id': '136a41cec2dd1e5121234a0453f1727b25a330e4'}, '3': {'artist': 'Alina Baraz', 'title': 'Take It Home', 'date': '16 Feb 2022, 00:04', 'album': 'It Was Divine', 'timestamp': '1644969890', 'id': '43560320d05b07f27ce54b6a3f0a1ae9a70dc7a1'}, '4': {'artist': 'Alina Baraz', 'title': 'The Beginning', 'date': '16 Feb 2022, 00:01', 'album': 'It Was Divine', 'timestamp': '1644969678', 'id': 'f2a8da77d4ea14e5ec67d257286fd8f6bc454a1f'}, '5': {'artist': 'Mariah the Scientist', 'title'

In [ ]:
recent_tracks_rdd = sc.parallelize(recent_tracks)

In [ ]:
recent_tracks_rdd.union(recent_tracks_rdd).collect()

[{'0': {'album': 'It Was Divine',
   'artist': 'Alina Baraz',
   'date': '16 Feb 2022, 00:17',
   'id': '997c665d10565914f05320254d0d46b46e734155',
   'timestamp': '1644970620',
   'title': 'More Than Enough'},
  '1': {'album': 'Urban Flora',
   'artist': 'Alina Baraz',
   'date': '16 Feb 2022, 00:12',
   'id': '64704ce20b0403804e51973749c0a7023bb5f1e4',
   'timestamp': '1644970377',
   'title': 'Make You Feel'},
  '2': {'album': 'It Was Divine',
   'artist': 'Alina Baraz',
   'date': '16 Feb 2022, 00:09',
   'id': '136a41cec2dd1e5121234a0453f1727b25a330e4',
   'timestamp': '1644970153',
   'title': 'Who Got Me'},
  '3': {'album': 'It Was Divine',
   'artist': 'Alina Baraz',
   'date': '16 Feb 2022, 00:04',
   'id': '43560320d05b07f27ce54b6a3f0a1ae9a70dc7a1',
   'timestamp': '1644969890',
   'title': 'Take It Home'},
  '4': {'album': 'It Was Divine',
   'artist': 'Alina Baraz',
   'date': '16 Feb 2022, 00:01',
   'id': 'f2a8da77d4ea14e5ec67d257286fd8f6bc454a1f',
   'timestamp': '164496

In [ ]:
json_track = source.get('track', 'Charli XCX', 'Vroom Vroom')

AttributeError: ignored

'{"test": "ciccio", "test1": 1}'